<a href="https://colab.research.google.com/github/mnijhuis-dnb/open_source_workshop/blob/master/Workshop_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas

### Open Source Workshop

In [1]:
import pandas as pd 
import numpy as np

### Data inlezen 

Zelf een dataframe maken

In [ ]:
raw_data_a = {
        'account_id': ['0', '1', '2', '3', '4'],
        'waarde': [5218, 268000, 30000, 415000, 235400],
        'aantal': [1, 1, 2, 2, 1]}
df_a = pd.DataFrame(raw_data_a, columns = ['account_id', 'waarde', 'aantal'])
df_a

Ingelezen data weergeven

In [8]:
import urllib.request
file = urllib. request. urlopen('https://raw.githubusercontent.com/mnijhuis-dnb/open_source_workshop/master/OSW_example/Kernindicatoren.csv')
# Alleen nodig omdat we vanuit GitHub data laden voor lokale bestanden kun je gewoon file vervangen door het bestandspath
df = pd.read_csv(file, delimiter=',', header='infer', index_col=None, skiprows=0, error_bad_lines=True)

Ingelezen data weergeven

In [ ]:
df.head(5)

kolom namen weergeven

In [ ]:
df.columns

specifieke kolom selecteren

In [ ]:
df['waarde'].tail(10)

In [ ]:
col = df.columns
df[col[4]].tail(10)

Data types weergeven

In [ ]:
df.dtypes

Data type wijzigen

Voor  wijzigen naar numerieke data gebruik pd.to_numeric(df[kolom], errors='coerce'), voor string gebruik df['kolom'].astype(str)

Voor format string zie: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
df['Periode '] = pd.to_datetime(df['Periode '],format='%Y-%m',errors='coerce')
df.head(5)

In [ ]:
df.dtypes

Spaties verwijderen

In [ ]:
df.columns = df.columns.str.strip()
df['Periode'].head(5)

Data inlezen uit Excel

In [ ]:
xls_file = pd.ExcelFile('kernindicatoren.xlsx')
df = pd.read_excel(xls_file,sheet_name='Data', usecols='A, B, C, D, E') #sheet_name=None for all sheets
df.head(5)

Data inlezen uit een JSON

In [ ]:
file = urllib. request. urlopen('https://raw.githubusercontent.com/mnijhuis-dnb/open_source_workshop/master/OSW_example/Kernindicatoren.json')
df = pd.read_json(file, orient='records')
df.columns = df.columns.str.strip()
#df[df.columns.drop(['waarde','Periode'])] = df[df.columns.drop(['waarde','Periode'])].str.strip()
df['Instrument'] = df['Instrument'].str.strip()
df['InstrumentSub'] = df['InstrumentSub'].str.strip()
df['Sector'] = df['Sector'].str.strip()
df.head(10)

### Data schonen 

Fouten uit data halen

Informatie over het dataframe ophalen

In [ ]:
df.info()

Cellen met ongeldige waardes identificeren

In [ ]:
df['waarde'] = pd.to_numeric(df['waarde'], errors='coerce')
df.isna().sum()

Cellen met ongeldige waardes verwijderen, voor het vullen van nan's gebruik df.fillna(x), waarin x de waarde voor de nan is

In [ ]:
df.dropna(how='any', thresh=2, axis=0, subset=['waarde','Instrument'], inplace=False).head(10)

NaN waardes interpoleren

In [ ]:
df.head(15).interpolate(method='spline', order=2, limit=5, limit_direction='both', limit_area='inside')

Identieke rijen verwijderen

In [ ]:
print("Aantal rijen voor verwijderen {}".format(len(df)))
df.drop_duplicates(inplace=True)
print("Aantal rijen na verwijderen {}".format(len(df)))

Kolommen of rijen laten vallen

In [ ]:
df.drop(columns=['waarde - 10'], inplace=True)
df.head(5)

Data afronden

In [ ]:
df.round({'waarde' : -2}).head(5)

Data discreet maken

In [ ]:
pd.cut(df['waarde'], bins=[0,1e1,1e2,1e3,1e4,1e5,1e6,1e7],precision =2, retbins=True,  duplicates='drop')[0].head(5)

### Data indexeren en combineren 

Data indexeren op basis van labels of een boolean index

In [ ]:
df.loc[df['InstrumentSub'].isna()]

In [ ]:
df.loc[df['Periode']=='2006-06']

Data selecteren aan de hand van een integer index

In [ ]:
df.iloc[45:55,:3]

Een enkel element selecteren

In [ ]:
df.at[48,'waarde']

Chain indexing, een subset van een subset van de data bekijken (niet gebruiken voor het toewijzen van data in een dataframe!)

In [ ]:
df.loc[df['Periode']=='2006-06'].loc[:,'waarde']

Meerdere filters combineren

In [ ]:
df.loc[(df['Instrument'].str.contains('hypotheken', case=False, na=False)) 
       & (df['InstrumentSub'].str.contains('zuiver nieuwe leningen', case=False, na=False)) 
       & (df['InstrumentSub'].str.contains('rente', case=False, na=False)) 
       & ((df['waarde'] >= 3) | (df['waarde'] < 2))]

De index opnieuw aanmaken

In [ ]:
df2 = df.set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)
df2.head(20)

De index gebruiken voor het selecteren

In [ ]:
df2.loc['Consumptief krediet',:,'Huishoudens','2006-06' ]

Index maken en aanvullen

In [ ]:
df['Periode'] = pd.to_datetime(df['Periode'],format='%Y-%m',errors='coerce')
df2 = df.set_index('Periode').head(20).reindex(pd.date_range('1/1/2002', periods=240, freq='MS'))
print(df2.loc['2002-05-01'])
print('')
print(df2.loc['2003-05-01'])

Kolom toevoegen

In [ ]:
df['waarde - 10'] = df['waarde'] - 10 
df.head(5)

Verschillende dataframes aan elkaar toevoegen 

In [ ]:
df1 = df.tail(10)
df2 = df.head(10)
df1.append(df2, ignore_index=False, sort=False)

Databronnen die samengevoegd moeten worden

In [ ]:
raw_data_a = {
        'account_id': ['0', '1', '2', '3', '4'],
        'datum': ['2009-12', '2010-01', '2009-12', '2010-01', '2010-01'],
        'type': ['Consumptief krediet', 'Woninghypotheken', 'Zakelijke kredietverlening', 'Woninghypotheken', 'Zakelijke kredietverlening'],
        'initieel bedrag': [5218, 268000, 30000, 415000, 235400]}
df_a = pd.DataFrame(raw_data_a, columns = ['account_id', 'datum', 'type', 'initieel bedrag'])
print(df_a)
print('')
raw_data_b = {
        'account_id': ['0', '0', '1', '3', '0', '2', '1', '3', '3', '2','5'],
        'datum': ['2009-12', '2010-01', '2010-01', '2010-01', '2010-01', '2010-01', '2010-02', '2010-02', '2010-02', '2010-02', '2010-02'],
        'afbetaling': [211, 516, 245, 189, 520, 110, 548, 328, 1065, 985, 1050],
        'type': ['standaard', 'standaard', 'standaard', 'standaard', 'extra', 'standaard', 'standaard', 'standaard', 'extra', 'standaard', 'standaard']}
df_b = pd.DataFrame(raw_data_b, columns = ['account_id', 'datum', 'afbetaling','type'])
print(df_b)

Samenvoegen van tabellen via merge

In [ ]:
pd.merge(df_a, df_b, how='outer', left_on=['account_id', 'datum'], right_on=['account_id', 'datum'],suffixes=('_a', '_b'),indicator=True)

De data uit twee dataframes bij elkaar optellen

In [ ]:
raw_data_a = {
        'account_id': ['0', '1', '2', '3', '4'],
        'waarde': [5218, 268000, 30000, 415000, 235400],
        'aantal': [1, 1, 2, 2, 1]}
df_a = pd.DataFrame(raw_data_a, columns = ['waarde', 'aantal'], index = raw_data_a['account_id'])
print(df_a)
print('')
raw_data_b = {
        'account_id': ['1', '0', '3', '2'],
        'waarde': [100, 200, 300, 400],
        'aantal': [1, 1, 1, 1]}
df_b = pd.DataFrame(raw_data_b, columns = ['waarde', 'aantal'], index = raw_data_b['account_id'])
print(df_b)
print(df_a+df_b)

### Data analyseren 

Data sorteren

In [ ]:
df.sort_values(by=['Periode','Sector'], ascending=True, na_position='last').tail(15)

De cijferdata uit een dataframe halen

In [ ]:
numerieke_waardes = df['waarde'].values
print(numerieke_waardes)

Basis statistieken voor de data weergeven (numerieke kolommen)

In [ ]:
df.describe()

of handmatig 

In [ ]:
print(df.quantile(q=0.9))
print(df.max())

Pivot tabel maken

In [ ]:
pd.pivot_table(df, values=['waarde'], index=['Periode', 'InstrumentSub'], aggfunc={'waarde': [min, max, sum]}).tail(20)

Maken van een lopend gemiddelde

In [ ]:
df_uitstaande = df[df['InstrumentSub']=='Uitstaande bedragen (mln euros)'].copy()
df_uitstaande['lopende gemiddelde'] = df_uitstaande['waarde'].rolling(6, min_periods=4).mean()
df_uitstaande.head(12)

Data combineren uit verschillende dataframes

In [ ]:
df_a = df_uitstaande.copy()
df_a = df_a.set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)

df_b = df_uitstaande.copy()
df_b = df_b.set_index(['Instrument','InstrumentSub','Sector','Periode'],append=False)
df_b['waarde'] = df_b['lopende gemiddelde']

df_a.combine(df_b, np.minimum, fill_value = 8).head(10)

Aantal keren tellen dat een waarde voorkomt

In [ ]:
df['waarde'].value_counts(bins=10,dropna=True,normalize=True)

Data grouperen

In [ ]:
df.groupby(['Instrument','InstrumentSub']).agg({'waarde': [np.min, np.max, np.mean]})

Berekeningen toepassen op elke rij/kolom

In [ ]:
df.apply(
    lambda x: [x['waarde'] + 3, 1] if x['Periode'] < pd.to_datetime("2015-01-01") else [x['waarde']-3, -1]
    , axis=1, raw=True, result_type='expand')

Over de rijen van een dataframe ittereren

In [ ]:
for index, row in df.iterrows():
    if index % 1000 == 0:
        df.at[index,'waarde'] = 0
        print(row)

Data plotten

In [ ]:
df['waarde'].plot()

Het package voor het plotten laden en de standaard figuurgroote aanpassen

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

Scatterplot maken

In [ ]:
df_a = df[df['Instrument']=='Woninghypotheken'].dropna()
df_a = df_a[df_a['InstrumentSub'].str.contains('Rente op uit')].set_index('Periode')

df_b = df[df['Instrument']=='Zakelijke kredietverlening'].dropna()
df_b = df_b[df_b['InstrumentSub'].str.contains('Rente op uit')].set_index('Periode')

df_c = pd.merge(df_b,df_a, how='inner', left_on=['Periode'], right_on=['Periode'], suffixes=('_c','_b'))
df_c.plot.scatter(x='waarde_c',y='waarde_b', alpha=0.5)

Bar plot maken

In [ ]:
df[df['Periode']=='2010-06'].plot.bar(y='waarde',x='InstrumentSub')

Histogram maken

In [ ]:
df.drop(columns=['Periode']).plot.hist(bins=20, density=False)

### Data wegschrijven 

Data die je in Python heb gebruikt wegschrijven om later weer te gebruiken of in andere programma's te gebruiken

Data wegschrijven om later te gebruiken

In [ ]:
df.to_pickle('results.pkl')

Data wegschrijven als een csv

In [ ]:
df.to_csv('results.csv', index=False, decimal=',', sep='\t')

Data wegschrijven naar Excel

In [ ]:
with pd.ExcelWriter('results.xlsx') as writer:  
    df[df['Instrument']=='Woninghypotheken'].to_excel(writer, sheet_name='Woninghypotheken')
    df[df['Instrument']=='Zakelijke kredietverlening'].to_excel(writer, sheet_name='Zakelijke kredietverlening')